In [3]:
# dependancies

import pandas as pd
import numpy as np
import math
from math import ceil, floor
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
import os
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import measure
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "browser"
import re


In [116]:
# visualise all nuclei

# read all nuclei coordinates into one large data frame and plot by color - assign label if possible

import pandas as pd
import glob

path = r'06022022_G05_multiOrganelles_Linked' # use your path
all_files = glob.glob(path + "/0_prediction_c0.*.csv")

li = []

for filename in all_files:
    labels = re.split('06022022_G05_multiOrganelles_Linked/0_prediction_c0.model.p_cell_|_coordinates.csv', filename)
    df = pd.read_csv(filename, index_col=None, header=0)
    if df.max(numeric_only=True).max() < -3000:
        print(df.max(numeric_only=True).max())
    df["label"] = int(labels[1])
    li.append(df)

nuclei_frame = pd.concat(li, axis=0, ignore_index=True)




In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=nuclei_frame["x"],
    y=nuclei_frame["y"],
    z=nuclei_frame["z"],
    mode='markers',
    marker=dict(
        size=2,
        color=nuclei_frame["label"],                # set color to an array/list of desired values
        colorscale='Viridis',
        opacity=0.8
    )
)])

fig.update_layout(scene_aspectmode='data')
fig.show()


In [117]:
# visualise all mito

import pandas as pd
import glob

path = r'06022022_G05_multiOrganelles_Linked' # use your path
all_files = glob.glob(path + "/0_target_mito*.csv")

print("hi", len(all_files))

li = []

for filename in all_files:
    labels = re.split('06022022_G05_multiOrganelles_Linked/0_target_mito_cell_|_coordinates.csv', filename)
    df = pd.read_csv(filename, index_col=None, header=0)
    df["label"] = int(labels[1])
    li.append(df)

mito_frame = pd.concat(li, axis=0, ignore_index=True)



hi 819


In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=mito_frame["x"],
    y=mito_frame["y"],
    z=mito_frame["z"],
    mode='markers',
    marker=dict(
        size=2,
        color=mito_frame["label"],                # set color to an array/list of desired values
        colorscale='Viridis',
        opacity=0.8
    )
)])

fig.update_layout(scene_aspectmode='data')
fig.show()

In [123]:
# visualise mito and nuclei together

#subset nuclei and mito <3

s_nuclei_frame = nuclei_frame.loc[nuclei_frame['label'] <= 10]
s_mito_frame = mito_frame.loc[mito_frame['label'] <= 10]

print(s_nuclei_frame.head())

fig = go.Figure()

trace1 = go.Scatter3d(
        x=s_nuclei_frame["x"],
        y=s_nuclei_frame["y"],
        z=s_nuclei_frame["z"],
        mode='markers',
        marker=dict(
            size=2,
            color=s_nuclei_frame["label"],                # set color to an array/list of desired values
            colorscale='Turbo',
            opacity=0.8, 
        ),
        name = 'nuclei'
    )
trace2 = go.Scatter3d(
        x=s_mito_frame["x"],
        y=s_mito_frame["y"],
        z=s_mito_frame["z"],
        mode='markers',
        marker=dict(
            size=2,
            color=s_mito_frame["label"],                # set color to an array/list of desired values
            colorscale='Viridis',
            opacity=0.8
        ),
        name = 'mito'
    )

fig.add_trace(trace1)
fig.add_trace(trace2)


fig.update_layout(scene_aspectmode='data')
fig.show()

         y    x  z  label
304036  49  365  0      4
304037  50  364  0      4
304038  46  365  1      4
304039  46  366  1      4
304040  47  365  1      4


In [8]:
# plot all nucleus files together 

class SliceGenerator():
    def __init__(self, image_path, image_size):
        self.image_path = image_path
        self.image_size = image_size

    def slice_image(self):
        ex_nuc = pd.read_csv(self.image_path)
        ex_nuc = ex_nuc.astype(float)

        minx = np.min(ex_nuc["x"])
        miny = np.min(ex_nuc["y"])

        image_base = np.zeros((self.image_size, self.image_size))
        x_padding = (self.image_size - (np.max(ex_nuc["x"]) - np.min(ex_nuc["x"])))/2
        y_padding = (self.image_size - (np.max(ex_nuc["y"]) - np.min(ex_nuc["y"])))/2
        x_left = ceil(x_padding)
        y_left = ceil(y_padding)

        ex_nuc["new_y"] = ex_nuc["y"] - miny + y_left
        ex_nuc["new_x"] = ex_nuc["x"] - minx + x_left

        image_set = []
        for i in set(ex_nuc["z"].values): #slice
            image_base = np.zeros((self.image_size, self.image_size))
            for row in range(0, len(ex_nuc["z"])-1): #index in ex_nuc
                if ex_nuc["z"].values[row] == i: #just focussing on slice by slice
                    if int(ex_nuc["new_x"].values[row]) >= self.image_size or int(ex_nuc["new_y"].values[row]) >= self.image_size:
                        print(int(ex_nuc["new_x"].values[row])-1,int(ex_nuc["new_y"].values[row])-1)
                    image_base[int(ex_nuc["new_x"].values[row])-1,int(ex_nuc["new_y"].values[row])-1] = 1 #0 index
                else:
                    pass
            image_set.append(image_base)
        
        return image_set

    def plot_image(self):
        image_set = self.slice_image()
        fig = plt.figure(figsize=(20, 20))
        for i in range(0, len(image_set)):
            img = image_set[i]
            plt.subplot(6,6,i+1)
            plt.axis("off")
            plt.imshow(img, cmap="gray")
    
    def unravel_slices(self):
        image_set = self.slice_image()
        unravelled = []
        for array in image_set:    
            unravelled.extend(np.ravel(array))
        
        return unravelled
    
    def plot_3D(self):

        image_set = self.slice_image()
        ex = np.stack(image_set)

        print(ex.shape)

        # Use marching cubes to obtain the surface mesh of these ellipsoids
        verts, faces, normals, values = measure.marching_cubes_lewiner(ex, 0)

        # Display resulting triangular mesh using Matplotlib. This can also be done
        # with mayavi (see skimage.measure.marching_cubes_lewiner docstring).
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection='3d')

        # Fancy indexing: `verts[faces]` to generate a collection of triangles
        mesh = Poly3DCollection(verts[faces])
        mesh.set_edgecolor('k')
        ax.add_collection3d(mesh)


        ax.set_xlabel("x-axis: a = 6 per ellipsoid")
        ax.set_ylabel("y-axis: b = 10")
        ax.set_zlabel("z-axis: c = 16")

        ax.set_xlim(0, self.image_size)  # a = 6 (times two for 2nd ellipsoid)
        ax.set_ylim(0, self.image_size)  # b = 10
        ax.set_zlim(0, self.image_size)  # c = 16


        plt.tight_layout()
        base = os.path.basename(os.path.normpath(self.image_path))
        plt.savefig(base+".png")
        plt.close(fig)

In [9]:
i = SliceGenerator("/Users/esthomas/Mirror/Andor_Rotation/github_repo/3D_cells/I05_multiOrganelles_Linked/nucleus.p_cell_1_coordinates.csv", 100)
i.plot_3D()

(68, 100, 100)


/var/folders/p1/99sndrn56lv71k57w35yh8n40000gn/T/ipykernel_25316/442201695.py:63: FutureWarning:

marching_cubes_lewiner is deprecated in favor of marching_cubes. marching_cubes_lewiner will be removed in version 0.19



In [10]:
from skimage import measure
from skimage.draw import ellipsoid


# Generate a level set about zero of two identical ellipsoids in 3D
ellip_base = ellipsoid(6, 10, 16, levelset=True)
ellip_double = np.concatenate((ellip_base[:-1, ...],
                               ellip_base[2:, ...]), axis=0)

In [17]:
ellip_double[15].shape

(23, 35)